In [1]:
%pip install pandas 
%pip install mysql-connector-python
%pip install decouple
%pip install pymysql sqlalchemy
%pip install python-dotenv



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
# Load environment variables from .env file (if you're using dotenv)
load_dotenv()

True

In [4]:
username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
database = os.getenv('DB_NAME') 

engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

In [5]:
class DatasetLoader:
    def load_data(self, filepath):
        return pd.read_csv(filepath, encoding = "ISO-8859-1", engine='python')

In [6]:
import pandas as pd
from sqlalchemy import create_engine

class DataCleaner:
    def handle_missing_values(self, data):
        print(data.isnull().sum())  # Count missing values per column
        data = data.dropna()  # Drop rows with any null values
        return data

    def check_header_data_types(self, data, data_types):
        for column, dtype in data_types.items():
            if column in data.columns:
                try:
                    data[column] = data[column].astype(dtype)
                except ValueError as e:
                    print(f"Error converting {column} to {dtype}: {e}")
        return data

    def validate_values(self, data):
        data_types = {
            "InvoiceNo": "object",
            "StockCode": "object",
            "Description": "object",
            "Quantity": "float64",
            "InvoiceDate": "datetime64[ns]",
            "UnitPrice": "float64",
            "CustomerID": "float64",
            "Country": "object"
        }
        # Step 1: Fix data types first (may introduce new NaNs)
        data = self.check_header_data_types(data, data_types)
        
        # Step 2: Remove duplicates based on key columns
        data = self.handle_duplicates(data, subset=["InvoiceNo", "StockCode"])
        return data


    def normalise_country_names(self, data):
        data['Country'] = data['Country'].replace('RSA', 'South Africa')
        data['Country'] = data['Country'].replace('EIRE', 'Ireland')

    def handle_duplicates(self, data, subset=None, keep='first'):
        """
        Remove duplicate rows and print a detailed report.
        """
        # Track initial state
        initial_rows = len(data)
        duplicate_mask = data.duplicated(subset=subset, keep=keep)
        duplicate_count = duplicate_mask.sum()
    
        # Print duplicate report
        print("\n=== Duplicate Report ===")
        print(f"Initial rows: {initial_rows}")
        print(f"Duplicate rows found: {duplicate_count}")
    
        if duplicate_count > 0:
            # Show example duplicates (first 2 rows)
            print("\nExample duplicate rows:")
            duplicates = data[duplicate_mask].head(2)
            print(duplicates.to_string(index=False))
    
        # Remove duplicates
        data_cleaned = data.drop_duplicates(subset=subset, keep=keep)
        final_rows = len(data_cleaned)
        print(f"\nRows after removal: {final_rows}")
        print(f"Total duplicates removed: {initial_rows - final_rows}")
    
        return data_cleaned

    def unique_check(self, data):
        for column in data.columns:
            if data[column].duplicated().any():
                print(f"Warning: Duplicate values found in column {column}")
        return data


    def separate_date_format(self, data):
        df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
        df['Date'] = df['InvoiceDate'].dt.date
        df['Time'] = df['InvoiceDate'].dt.time

    def save_changes(self, data):
        data.to_csv('../dataset/cleaned_data.csv', index=False)
        print("Cleaned data saved successfully!")
        return data

    def save_to_sql(self, data, table_name):
        username = os.getenv('DB_USERNAME')
        password = os.getenv('DB_PASSWORD')
        host = os.getenv('DB_HOST')
        database = os.getenv('DB_NAME')

        engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')
        data.to_sql(table_name, con=engine, if_exists='replace', index=False)
        print(f"Data saved to {table_name} table successfully!")


In [7]:
class ExploreData:
    def total_revenue(self, data):
        data['TotalSold'] = data['Quantity'] * data['UnitPrice']
        total_revenue = "${:,.2f}".format(data['TotalSold'].sum())
        return total_revenue


    def sales_by_country(self, data):
        sales_by_country = data.groupby('Country')['TotalSold'].sum().reset_index()
        return sales_by_country

    
    def singapore(self, data):
        # Filter the DataFrame for rows where the 'Country' is 'Singapore'
        singapore_data = data[data['Country'] == 'Singapore']
        
        # Calculate the mean UnitPrice
        # average_price = singapore_data['UnitPrice']
        
        sns.boxplot(x='UnitPrice', data=singapore_data)
        plt.title('Price Box Plot in Singapore')
        plt.show()


        singapore_data = singapore_data[['UnitPrice']].median()
        
        return singapore_data


    
    
    def average_price_by_country(self, data):
        average_price_by_country = data.groupby('Country')['UnitPrice'].median().reset_index()
        
        # Rename columns
        average_price_by_country = average_price_by_country.rename(columns={
            'Country': 'Country',
            'UnitPrice': 'AveragePrice'
        })

        average_price_by_country['AveragePrice'] = average_price_by_country['AveragePrice'].map('{:.2f}'.format)
        # Extract data for the pie chart
        labels = data['Country']
        sizes = data['UnitPrice']
        
        # Create the pie chart
        plt.figure(figsize=(8, 6))
        plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
        
        # Equal aspect ratio ensures that pie is drawn as a circle
        plt.axis('equal')
        
        # Title
        plt.title('Sales by Country')
        
        # Display the pie chart
        plt.show()
        
        return average_price_by_country

In [8]:

if __name__ == "__main__":
    # Load your data
    df = pd.read_csv('../dataset/data.csv')

    
    # Instantiate DataCleaner and clean your data
    cleaner = DataCleaner()
    df = cleaner.handle_missing_values(df)
    df = cleaner.validate_values(df)
    df = cleaner.unique_check(df)
    

    
    

    

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

=== Duplicate Report ===
Initial rows: 396681
Duplicate rows found: 0

Rows after removal: 396681
Total duplicates removed: 0


In [9]:
    df.describe()
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396681 entries, 0 to 396680
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    396681 non-null  object        
 1   StockCode    396681 non-null  object        
 2   Description  396681 non-null  object        
 3   Quantity     396681 non-null  float64       
 4   InvoiceDate  396681 non-null  datetime64[ns]
 5   UnitPrice    396681 non-null  float64       
 6   CustomerID   396681 non-null  float64       
 7   Country      396681 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 24.2+ MB


In [10]:
    cleaner.save_changes(df)

Cleaned data saved successfully!


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE META,6.0,2010-01-12 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-01-12 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
396676,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12.0,2011-09-12 12:50:00,0.85,12680.0,France
396677,581587,22899,CHILDREN'S APRON DOLLY GIRL,6.0,2011-09-12 12:50:00,2.10,12680.0,France
396678,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4.0,2011-09-12 12:50:00,4.15,12680.0,France
396679,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4.0,2011-09-12 12:50:00,4.15,12680.0,France


The total revenue 

In [12]:
    explorer = ExploreData()
    total_revenue = explorer.total_revenue(df)
    print(f"Total Revenue: {total_revenue}")

Total Revenue: $8,246,367.30


The sales by country

In [14]:
    sale_by_country = explorer.sales_by_country(df)
    print(f"Sales by countries: {sale_by_country}")

Sales by countries:                  Country    TotalSold
0              Australia   136976.570
1                Austria    10154.320
2                Bahrain      548.400
3                Belgium    40910.960
4                 Brazil     1143.600
5                 Canada     3666.380
6        Channel Islands    20076.390
7                 Cyprus    12726.760
8         Czech Republic      707.720
9                Denmark    18726.890
10                  EIRE   249985.280
11    European Community     1291.750
12               Finland    22125.140
13                France   196516.480
14               Germany   221004.020
15                Greece     4710.520
16               Iceland     4310.000
17                Israel     6988.400
18                 Italy    16890.510
19                 Japan    35340.620
20               Lebanon     1693.880
21             Lithuania     1661.060
22                 Malta     2505.470
23           Netherlands   284661.540
24                Norway    34

Total sales by season, month

In [ ]:
    average_price_by_country = explorer.average_price_by_country(df)
    print(average_price_by_country)

In [ ]:
    singa = explorer.singapore(df)
    print(singa)

In [ ]:
    singa = explorer.france(df)
    print(singa)